### Scraping sustainable brand Everlane

In [12]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import urllib.parse

In [6]:
# jeans
base_url = "https://www.everlane.com/collections/womens-jeans?page={}"

link_list = []
driver = webdriver.Chrome()

try:
    for page_num in range(1, 14):
        url = base_url.format(page_num)
        driver.get(url)
        time.sleep(2)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        links = soup.find_all('a', {'role': 'link'})
        
        for link in links:
            href = link.get('href')
            if href:
                link_list.append(href)
                
        print(f"Page {page_num} scraped")
        
finally:
    driver.quit()
    


Page 1 scraped
Page 2 scraped
Page 3 scraped
Page 4 scraped
Page 5 scraped
Page 6 scraped
Page 7 scraped
Page 8 scraped
Page 9 scraped
Page 10 scraped
Page 11 scraped
Page 12 scraped
Page 13 scraped


In [8]:
print("Total links collected:", len(link_list))

Total links collected: 93


In [10]:
unique_links = list(set(link_list))
len(unique_links)

60

In [13]:
base = "https://www.everlane.com/"
unique_full_urls = []
for href in unique_links:
    absolute_url = urllib.parse.urljoin(base, href)
    unique_full_urls.append(absolute_url)

In [41]:
u = unique_full_urls[0]
driver = webdriver.Chrome()
try:
    driver.get(u)
    time.sleep(2)

    try:
        button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "styles_product-details__expand__fxMdk"))
        )
        time.sleep(2)
        button.click()
        time.sleep(2)
    except Exception as e:
        print(f"Error clicking button")
        
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    product_name_elem = soup.find('div', class_='styles_product-header__title-name__jqMTS')

    if product_name_elem:
        product_name = product_name_elem.get_text(strip=True)
    else:
        product_name = "Product Name Not Found"
        
    price_elem = soup.find('div', class_='styles_product-header__price__17H6S')
    if price_elem:
        product_price = price_elem.get_text(strip=True)
    else:
        product_price = "Price Not Found"
        
    materials = "Materials Not Found"
    slide_down_div = soup.find('div', class_='styles_slide-down-up__vEboW styles_product-details--expanded__Du6z4 slide-down-up-enter-done')
    if slide_down_div:
        # first child div within the slide-down div
        first_child_div = slide_down_div.find('div', recursive=False)
        if first_child_div:
            prod_details_div = first_child_div.find('div', class_='styles_product-details-item__detail__We_Zc')
            if prod_details_div:
                materials = prod_details_div.get_text(strip=True)

finally:
    driver.quit()

Error clicking button


In [44]:
product_name

'The Original Curvy Cheeky® Jean'

In [16]:
products_info = []
driver = webdriver.Chrome()

try:
    for url in unique_full_urls[0]:
        driver.get(url)
        time.sleep(2)
        
        # click the button for details
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "styles_product-details__expand__fxMdk"))
            )
            button.click()
            time.sleep(2)
        except Exception as e:
            print(f"Error clicking button")
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        product_name_elem = soup.find('div', class_='styles_product-header__title-name__jqMTS')
        
        if product_name_elem:
            name_meta = product_name_elem.find('meta', itemprop='name')
            product_name = name_meta['content']
        else:
            product_name = "Product Name Not Found"
            
        price_elem = soup.find('div', class_='product-heading__price')
        if price_elem:
            price_meta = price_elem.find('meta', itemprop='highPrice')
            if price_meta and price_meta.get('content'):
                product_price = price_meta['content']
            else:
                product_price = "Price Not Found"
        else:
            product_price = "Price Div Not Found"
            
        
        materials = "Materials Not Found"
        slide_down_div = soup.find('div', class_='styles_slide-down-up__vEboW styles_product-details--expanded__Du6z4 slide-down-up-enter-done')
        if slide_down_div:
            # first child div within the slide-down div
            first_child_div = slide_down_div.find('div', recursive=False)
            if first_child_div:
                prod_details_div = first_child_div.find('div', class_='styles_product-details-item__detail__We_Zc')
                if prod_details_div:
                    materials = prod_details_div.get_text(strip=True)
        
        
        factory_div = soup.find('div', class_='styles_product-details-item__title__N4LsX', string='Factory')
        if factory_div:
            factory_next_div = factory_div.find_next_sibling('div', style='width: 100%;')
            if factory_next_div:
                factory_content = factory_next_div.find('div').get_text(strip=True)
                factory_link = factory_next_div.find('a')['href']
            else:
                factory_content = "Factory Content Not Found"
                factory_link = "Factory Link Not Found"
        
        else:
            details_div = soup.find('div', class_='styles_product-details-item__title__N4LsX', string='Details')
            if details_div:
                details_next_div = details_div.find_next_sibling('div', style='width: 100%;')
                if details_next_div:
                    factory_content = details_next_div.find('div').get_text(strip=True)
                    factory_link = "Factory Link Not Found"
                else:
                    factory_content = "Factory Content Not Found"
                    factory_link = "Factory Link Not Found"
            else:
                factory_content = "Factory Content Not Found"
                factory_link = "Factory Link Not Found"
        
    

        products_info.append((product_name, product_price, factory_content, factory_link))
        print(f"Collected product info for {product_name}: {product_price} | Materials: {materials} | Factory: {factory_content} | Factory Link: {factory_link}")
        
except Exception as e:
    print(f"Error occurred: {e}")

finally:
    driver.quit()

Error occurred: Message: invalid argument
  (Session info: chrome=124.0.6367.80)
Stacktrace:
0   chromedriver                        0x00000001047b69c8 chromedriver + 5216712
1   chromedriver                        0x00000001047ae603 chromedriver + 5182979
2   chromedriver                        0x000000010432036b chromedriver + 406379
3   chromedriver                        0x0000000104307091 chromedriver + 303249
4   chromedriver                        0x0000000104305bc2 chromedriver + 297922
5   chromedriver                        0x0000000104305e9a chromedriver + 298650
6   chromedriver                        0x0000000104322ebf chromedriver + 417471
7   chromedriver                        0x00000001043ae255 chromedriver + 987733
8   chromedriver                        0x000000010438e702 chromedriver + 857858
9   chromedriver                        0x00000001043ad6bf chromedriver + 984767
10  chromedriver                        0x000000010438e4a3 chromedriver + 857251
11  chromedriv

In [ ]:
print("Total product names collected:", len(product_names))